In [1]:
import os
import time
from tqdm import tqdm
import numpy as np
import pandas as pd

from selenium import webdriver

In [2]:
# List of 20 codes to collect the data

code_list = ['005380', '033780', '000270', '207940', '005490', '006400',
       '373220', '096770', '032830', '035420', '028260', '034730',
       '012330', '005935', '066570', '015760', '003670', '055550',
       '105560', '051910']

In [3]:
# Selenium Chromedriver configurations

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

chrome_driver_dir = os.path.join(os.getcwd(),'chromedriver')
driver = webdriver.Chrome(chrome_driver_dir, options = chrome_options)

In [ ]:
for code in code_list:
    all_list = []
    
    # Access the Board page of the corresponding code
    driver.get(f'https://finance.naver.com/item/board.naver?code={code}')
    time.sleep(0.5)

    # Find out the maxiumum page number, in order to set the range of the for loop
    last_page_button = driver.find_element_by_xpath('//*[@id="content"]/div[2]/table[2]/tbody/tr/td[2]/table/tbody/tr/td[12]/a')
    last_page_button.click()
    page_num = int(driver.current_url.split('&page=')[1])


    # Iterate through each page, and get the datetime, title, up vote, and down vote for each thread.s
    for page in tqdm(range(1, page_num)):
        driver.get(f'https://finance.naver.com/item/board.naver?code={code}&page={page}')
        
        for i in range(3,26):
            if i in [8, 14, 20]: 
                continue
            up_count = int(driver.find_element_by_xpath(f'//*[@id="content"]/div[2]/table[1]/tbody/tr[{i}]/td[5]/strong').text)
            down_count = int(driver.find_element_by_xpath(f'//*[@id="content"]/div[2]/table[1]/tbody/tr[{i}]/td[6]/strong').text)
            
            if up_count > down_count:
                title = driver.find_element_by_xpath(f'//*[@id="content"]/div[2]/table[1]/tbody/tr[{i}]/td[2]/a').text
                datetime = driver.find_element_by_xpath(f'//*[@id="content"]/div[2]/table[1]/tbody/tr[{i}]/td[1]/span').text
                post_list = [datetime, title, up_count, down_count]
                all_list.append(post_list)
                
    all_df = pd.DataFrame(all_list,columns=['datetime','title','up','down'])
    all_df.to_csv(os.path.join(os.getcwd(),'raw',f'{code}.csv'),index=False)
    print(f'{code}.csv successfully saved')